# `prachathai-67k` Classification Benchmark

We provide two benchmarks for 12-topic multi-label classification of `prachathai-67k`: [fastText](https://github.com/facebookresearch/fastText) and [ULMFit](https://github.com/cstorm125/thai2fit). In both cases, we first finetune the embeddings using all data. The data is then split into train, validation and test sets at 70/10/20 split. The benchmark numbers are based on the test set. Performance metrics are macro-averaged accuracy, F1 score, precision and recall.

In [1]:
import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from ast import literal_eval
from tqdm import tqdm_notebook
from collections import Counter
import re

#viz
from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns

benchmark_labels = ['การเมือง','สิทธิมนุษยชน','คุณภาพชีวิต','ต่างประเทศ','สังคม',
                  'สิ่งแวดล้อม','เศรษฐกิจ','วัฒนธรรม','แรงงาน','ความมั่นคง','ไอซีที','การศึกษา']

def replace_newline(t):
    return re.sub('[\n]{1,}', ' ', t)

ft_data = 'ft_data/'

In [2]:
prachathai = pd.read_csv('prachathai-67k.csv')
print(prachathai.shape)
prachathai.head()

(67889, 5)


,url,date,title,body_text,labels
0,https://prachatai.com/print/42,2004-08-24 14:31,"นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ""",ประชาไท --- 23 ส.ค.2547 นักวิจัยฯ ชี้นโยบายจี...,"['ข่าว', 'สิ่งแวดล้อม']"
1,https://prachatai.com/print/41,2004-08-24 14:22,ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ,ประชาไท- 23 ส.ค.2547 นักวิชาการ ภาคประชาชน จ...,"['ข่าว', 'สิ่งแวดล้อม']"
2,https://prachatai.com/print/43,2004-08-24 15:17,จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย,นโยบายที่อนุญาตให้ปลูกร่วมกับพืชอื่นได้นั้นถื...,"['ข่าว', 'สิ่งแวดล้อม']"
3,https://prachatai.com/print/45,2004-08-24 15:58,ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...,ประชาไท -- 23 ส.ค. 47 ขาประจำทักษิณ ฟันธง ฟอง...,"['ข่าว', 'การเมือง', 'คณะเศรษฐศาสตร์ มหาวิทยาล..."
4,https://prachatai.com/print/47,2004-08-24 16:10,กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง,ประชาไท-23 ส.ค.47 คณะกรรมการอนุรักษ์ ผืนป่าตะ...,"['ข่าว', 'สิ่งแวดล้อม']"


## Train-validation-test Split

We perform 70/10/20 train-validation-test split. The train-validation vs test split is done randomly and the train vs validation split is done using `MultilabelStratifiedShuffleSplit` as implemented by [iterative-stratification](https://github.com/trent-b/iterative-stratification). We will use the test set for classification benchmark.

In [3]:
all_df = prachathai[['body_text', 'labels']].copy()
all_df.head()

#use only benchmark labels
for lab in benchmark_labels:
    all_df[lab] = all_df.labels.map(lambda x: 1 if lab in x else 0)
all_df.drop('labels',axis=1,inplace=True)

In [4]:
all_df.head()

,body_text,การเมือง,สิทธิมนุษยชน,คุณภาพชีวิต,ต่างประเทศ,สังคม,สิ่งแวดล้อม,เศรษฐกิจ,วัฒนธรรม,แรงงาน,ความมั่นคง,ไอซีที,การศึกษา
0,ประชาไท --- 23 ส.ค.2547 นักวิจัยฯ ชี้นโยบายจี...,0,0,0,0,0,1,0,0,0,0,0,0
1,ประชาไท- 23 ส.ค.2547 นักวิชาการ ภาคประชาชน จ...,0,0,0,0,0,1,0,0,0,0,0,0
2,นโยบายที่อนุญาตให้ปลูกร่วมกับพืชอื่นได้นั้นถื...,0,0,0,0,0,1,0,0,0,0,0,0
3,ประชาไท -- 23 ส.ค. 47 ขาประจำทักษิณ ฟันธง ฟอง...,1,0,0,0,0,0,0,0,0,0,0,0
4,ประชาไท-23 ส.ค.47 คณะกรรมการอนุรักษ์ ผืนป่าตะ...,0,0,0,0,0,1,0,0,0,0,0,0


In [11]:
#randomly create test set
idx = np.arange(all_df.shape[0])
np.random.seed(1412)
np.random.shuffle(idx)
train_valid_idx, test_idx = idx[:int(all_df.shape[0] * 0.8)], idx[int(all_df.shape[0] * 0.7):]
print(len(train_valid_idx),len(test_idx))

#save test_df
test_df = all_df.iloc[test_idx,:]
print(test_df.shape)
test_df.to_csv('test_df.csv',index=False)

54311 20367
(20367, 13)


In [12]:
#stratified shuffle split for validation set
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

train_valid_df = all_df.iloc[train_valid_idx,:]
ohe_df = train_valid_df.iloc[:,1:]
msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.125, random_state=1412)
train_idx, valid_idx = next(msss.split(ohe_df,ohe_df))
print(len(train_idx),len(valid_idx))

#save train and valid_df
train_df = all_df.iloc[train_idx,:]
print(train_df.shape)
train_df.to_csv('train_df.csv',index=False)

valid_df = all_df.iloc[valid_idx,:]
print(valid_df.shape)
valid_df.to_csv('valid_df.csv',index=False)

47522 6789
(47522, 13)
(6789, 13)


In [43]:
#test set prevalence
test_df.iloc[:,1:].mean(0)

การเมือง        0.579614
สิทธิมนุษยชน    0.223450
คุณภาพชีวิต     0.169343
ต่างประเทศ      0.123337
สังคม           0.130505
สิ่งแวดล้อม     0.117543
เศรษฐกิจ        0.075907
วัฒนธรรม        0.063289
แรงงาน          0.056366
ความมั่นคง      0.056317
ไอซีที          0.041489
การศึกษา        0.037512
dtype: float64

## [fastText](https://github.com/facebookresearch/fastText) Model

We used embeddings pretrained on [Thai Wikipedia Dump](https://github.com/facebookresearch/fastText/blob/master/docs/pretrained-vectors.md) and finetuned them using all of `prachathai-67k` using skipgram model. After that, we do 12 binary classifications and compute macro-average of performance metrics.

| topic         | prevalence | accuracy | f1     | precision | recall |
|---------------|------------|----------|--------|-----------|--------|
| การเมือง       | 0.5796     | 0.8474   | 0.8731 | 0.8427    | 0.9058 |
| สิทธิมนุษยชน     | 0.2235     | 0.8846   | 0.7018 | 0.8299    | 0.6080 |
| คุณภาพชีวิต      | 0.1693     | 0.8976   | 0.6412 | 0.7887    | 0.5402 |
| ต่างประเทศ     | 0.1233     | 0.9523   | 0.7883 | 0.8713    | 0.7197 |
| สังคม          | 0.1305     | 0.8833   | 0.2933 | 0.7003    | 0.1855 |
| สิ่งแวดล้อม      | 0.1175     | 0.9469   | 0.7330 | 0.8967    | 0.6199 |
| เศรษฐกิจ       | 0.0759     | 0.9394   | 0.4436 | 0.7321    | 0.3182 |
| วัฒนธรรม       | 0.0633     | 0.9473   | 0.3569 | 0.7822    | 0.2312 |
| แรงงาน        | 0.0564     | 0.9726   | 0.7060 | 0.8933    | 0.5836 |
| ความมั่นคง      | 0.0563     | 0.9546   | 0.3990 | 0.7832    | 0.2677 |
| ไอซีที          | 0.0415     | 0.9728   | 0.5745 | 0.8184    | 0.4426 |
| การศึกษา       | 0.0375     | 0.9631   | 0.1237 | 0.5699    | 0.0694 |
| macro_average | 0.1396     | 0.9302   | 0.5529 | 0.7924    | 0.4576 |

In [8]:
train_df = pd.read_csv('train_df.csv')
valid_df = pd.read_csv('valid_df.csv')
test_df = pd.read_csv('test_df.csv')

In [6]:
df_txts = ['train','valid','test']
dfs = [train_df,valid_df,test_df]

for i in range(3):
    df = dfs[i]
    for lab in tqdm_notebook(benchmark_labels):
        ft_lines = []
        for _,row in df.iterrows():
            ft_lab = f'__label__{row[lab]}'
            ft_text = replace_newline(f'{row["body_text"]}')
            ft_line = f'{ft_lab} {ft_text}'
            ft_lines.append(ft_line)

        doc = '\n'.join(ft_lines)
        with open(f'{ft_data}{df_txts[i]}_{lab}.txt','w') as f:
            f.write(doc)
        f.close()

In [16]:
#for fasttext embedding finetuning
ft_lines = []
for _,row in all_df.iterrows():
    ft_lab = '__label__0'
    ft_text = replace_newline(f'{row["body_text"]}')
    ft_line = f'{ft_lab} {ft_text}'
    ft_lines.append(ft_line)

doc = '\n'.join(ft_lines)
with open(f'{ft_data}df_all.txt','w') as f:
    f.write(doc)
f.close()

In [18]:
#finetune with all data
!/home/ubuntu/fastText/fasttext skipgram \
-pretrainedVectors 'model/wiki.th.vec' -dim 300 \
-input ft_data/df_all.txt -output 'model/finetuned'

Read 16M words
Number of words:  161169
Number of labels: 1
Progress: 100.0% words/sec/thread:    8476 lr:  0.000000 loss:  1.250565 ETA:   0h 0m


In [38]:
#train classifier
!/home/ubuntu/fastText/fasttext supervised \
-input 'ft_data/train_การศึกษา.txt' -output 'model/การศึกษา' \
-pretrainedVectors 'model/finetuned.vec' -epoch 5 -dim 300 -wordNgrams 2 

Read 11M words
Number of words:  5773818
Number of labels: 2
Progress: 100.0% words/sec/thread:  276088 lr:  0.000000 loss:  0.027403 ETA:   0h 0m
^C


In [41]:
#get prediction
preds = !/home/ubuntu/fastText/fasttext predict 'model/การศึกษา.bin' 'ft_data/test_การศึกษา.txt'

In [42]:
#calculate score
lab = 'การศึกษา'
pred_num = [int(i[-1]) for i in preds]

from sklearn.metrics import f1_score, precision_score, recall_score
(pred_num == test_df[lab]).mean(), \
f1_score(test_df[lab],pred_num), \
precision_score(test_df[lab],pred_num), \
recall_score(test_df[lab],pred_num)

(0.9631266264054598,
 0.12368728121353562,
 0.5698924731182796,
 0.0693717277486911)

## [ULMFit](https://github.com/cstorm125/thai2fit) Model